In [1]:
import pandas as pd
import re
from bs4 import BeautifulSoup

In [2]:
data_path = 'D:/2021-09-04 ~ 2023-09-04 QNA DATA.csv'
# nrows = 읽어올 row 갯수
df = pd.read_csv(data_path)
df.columns = ['Board_idx', 'QA', 'Content', 'Writeday']
df[200:300]

,Board_idx,QA,Content,Writeday
200,4,A,<p>안녕하세요!</p><p>디딤365(운영위탁사) 운영센터입니다.</p><p><b...,2022-07-01 17:10:00
201,4,A,<p>안녕하세요 비즈데이터입니다.</p><p>보내주신 디딤365_CSAP 인증 FA...,2022-07-19 12:06:00
202,4,A,<p>안녕하세요!</p><p>디딤365(운영위탁사) 정보보안센터입니다.</p><p>...,2022-07-19 16:54:00
203,4,A,<p>테스트<br></p>,2022-09-15 15:38:00
204,4,A,<p>안녕하세요!</p><p>디딤365 운영센터입니다.</p><p><br></p><...,2022-09-15 15:43:00
...,...,...,...,...
295,5,Q,"<p><font face=""돋움"">안녕하세요. 메디오피아테크 윤성호 입니다.</fo...",2023-08-03 16:22:00
296,5,A,"<p style=""line-height: 15.6px; color: rgb(103,...",2023-08-03 16:38:00
297,5,A,<p>안녕하세요.&nbsp;</p><p>디딤365 운영센터입니다.&nbsp;<br>...,2023-08-07 11:48:00
298,5,A,<p>skforest 에스케이임업 7월분 세금계산서 발행 부탁드립니다.<br></p>,2023-08-09 10:36:00


In [69]:
# Removing Html Tags in Content (Take 5.7s)
def remove_html_tags(text):
    soup = BeautifulSoup(text, 'html.parser')
    return soup.get_text()

df['Content'] = df['Content'].apply(remove_html_tags)

C:\Users\이규하\AppData\Roaming\Python\Python38\site-packages\bs4\__init__.py:216: UserWarning: "b'.'" looks like a filename, not markup. You should probably open this file and pass the filehandle into Beautiful Soup.
  warnings.warn(


In [73]:
df

,Board_idx,QA,Content,Writeday
0,1,A,안녕하세요 파트너사인 시큐인프라 유영채 입니다.파트너사 변경으로 이메일 주소가 변경...,2021-09-08 09:53:00
1,1,A,안녕하세요!디딤365(운영위탁사) 운영센터입니다.요청하신 모니터링 알람 수신자의 메...,2021-09-08 14:30:00
2,1,A,안녕하세요 디딤365 입니다.(주)파슨텍 08월 정기보고서 전달 드립니다.압축 패스...,2021-09-10 14:01:00
3,1,A,문제 URL : http://coop88.hosting2003.co.kr해당 위 U...,2021-09-10 17:39:00
4,1,A,안녕하세요!디딤365(운영위탁사) 운영센터입니다..htaccess 파일을 통해 sh...,2021-09-10 18:00:00
...,...,...,...,...
119651,52602,A,안녕하세요!디딤365 운영센터입니다.담당자 확인 시 게시글 잘못 올린 것으로 확인하...,2023-09-03 14:44:00
119652,52603,A,"안녕하세요. 윤디스크 박승미입니다.웹서버 ACG에 80,443 정책을 추가하였습니다...",2023-09-03 18:34:00
119653,52603,A,안녕하세요!디딤365 운영센터입니다.오늘 사이트 오픈으로 인해 모니터링 정책 추가 ...,2023-09-03 18:50:00
119654,52603,Q,안녕하세요. 윤디스크 박승미입니다.클라우드 서버 이관 오픈이 완료되어 당분간 서버에...,2023-09-03 19:04:00


In [70]:
def pair_qna_data(input_df):
    # Output Dataframe
    new_df = pd.DataFrame(columns=['Board_idx', 'Count', 'Question', 'Answer'])

    q_count = 1
    current_idx = '1'

    q_content = ''
    a_content = ''

    for index, row in input_df.iterrows():
        if row['QA'] == 'Q':
            if current_idx == str(row['Board_idx']):
                if a_content.isspace() or a_content == '':
                    q_content += (' ' + row['Content'])
                else:
                    list = [current_idx, q_count, q_content, a_content]
                    new_df.loc[len(new_df)] = list
                    q_count += 1
                    q_content = row['Content']
                    a_content = ''
            else:
                q_count = 1
                q_content = row['Content']
                current_idx = str(row['Board_idx'])
                a_content = ''
        else:
            if q_content.isspace() or q_content == '':
                continue
            else:
                if current_idx == str(row['Board_idx']):
                    if a_content.isspace() or a_content == '':
                        a_content = row['Content']
                    else:
                        a_content += (' ' + row['Content'])
                else:
                    continue
    return new_df



        

In [71]:
new_df = pair_qna_data(df)

In [72]:
new_df

,Board_idx,Count,Question,Answer
0,1,1,안녕하세요 운영자님말씀주신 사항 이해하였습니다. 하기와 같이 요청드리오니 변경 부...,안녕하세요!디딤365(운영위탁사) 운영센터입니다.작업 후 게시판을 통해 회신 드리겠...
1,1,2,안녕하세요 운영자님 @ web2gether-db01​- 최소사양인 2vCPU 4GB...,안녕하세요!디딤365(운영위탁사) 운영센터입니다.내일 새벽 00시 30분에 작업 진...
2,1,3,지난번 발급받은 계정이 여전히 이용 가능한 상태임을 확인했습니다.번거롭게 해드려 죄...,안녕하세요!디딤365(운영위탁사) 운영센터입니다.계정 추가 발급이나 DidimSaa...
3,1,4,hng.yna.co.kr hng.yna.co.kr,test
4,1,5,test,안녕하세요.디딤365 운영센터입니다.고객님께서 사용 중이신 자다라 스토리지(NAS)...
...,...,...,...,...
9050,52480,1,참고로 9월 1일 부터 기본소득접수가 있어서업체분께서 데이터 동기화를 꼭 잘부탁...,"안녕하세요!디딤365 운영센터입니다.요청하신 apply-was, apply-admi..."
9051,52480,2,안녕하세요.이두리 입니다. 혹시 내일 데이터 동기화는 처음부터 한번 쭉 훑어서 전체...,안녕하세요!디딤365 운영센터입니다.문의주신 내용 관련하여 내일 주간 업무시간에 추...
9052,52504,1,조치 완료 되었고 내역 올려드리기 위해 작성하였습니다.,안녕하세요!디딤365 운영센터입니다.이력 공유 감사합니다.게시글 완료 처리 하겠습니...
9053,52513,1,점검 결과 에러로그 등 메시지 캡쳐 부탁드립니다.,안녕하세요!디딤365 운영센터입니다.아래와 같이 6대 서버가 Tomcat 사용 중으...
